In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import gmean

#known IU PFU input for Arms 4,5,6
DP_Titer = 3.1e11 #PFU/mL
dose_vol = 12 #mL
input_PFU = DP_Titer * dose_vol #3.72e12 PFU

#worth noting that the stated dose is ~2e12 PFU

#urine volumes
pkvol_v1v3 = pd.read_csv("pcmt12.CSV")
pkvol_v1v5 = pd.read_csv("pcmt2.CSV")

pkvol = pd.concat([pkvol_v1v3, pkvol_v1v5])

#merge urine vols with subject arms
subject_arms = pd.read_csv("subject_arms.tsv", sep="\t")
subject_arms.rename(columns={"subject_id" : "Subject"}, inplace=True)

pkvol_arms = pd.merge(pkvol, subject_arms, how='left')

#drop arms 1, 2, 3
pkvol_arms = pkvol_arms[(pkvol_arms.treatment_arm == "Arm 4") | (pkvol_arms.treatment_arm == "Arm 5") | (pkvol_arms.treatment_arm == "Arm 6")]

#subset to relevant columns
columns_subset = ["Subject", "instanceId", "InstanceName", "DataPageId", "RecordId","PCSPEC1","PCTPT8","PCTPT8_STD","PKVOL","PCDAT_RAW", "treatment_arm"]
pkvol_subset = pkvol_arms[columns_subset]

#merge with phage titers
titers = pd.read_csv("phage_titer_table.tsv", sep="\t")
titers.rename(columns = {"subject_id" : "Subject"}, inplace=True)
titers = titers[titers.sample_type == 'urine']
titer_col_subset = ["Subject", "visit", "day", "time_point", "b3383", "b3384", "b3830"]
titers_sub = titers[titer_col_subset]

#take max titer of the three strains
def max_titer(row):
    if (row["b3383"] >= row["b3384"]) & (row["b3383"] >= row["b3830"]):
        return row["b3383"]
    elif row["b3384"] >= row["b3830"]:
        return row["b3384"]
    else:
        return row["b3830"]

titers_sub["PFU/mL"] = titers_sub.apply(max_titer, axis=1)

titers_sub.drop(columns = ['visit',"b3383", "b3384", "b3830"], inplace=True)
titers_sub = titers_sub[titers_sub.day <=3]

print(titers_sub[titers_sub["time_point"] == 0]["PFU/mL"].mean())

#modify pkvol_subset to merge titers in

#get timepoints aligned
pkvol_subset["PCTPT8_STD"]=pkvol_subset["PCTPT8_STD"] -1
pkvol_subset.loc[pkvol_subset["PCTPT8_STD"] == 3, "PCTPT8_STD"] = 4
pkvol_subset.rename(columns = {"PCTPT8_STD" : "time_point"}, inplace=True)

#get visits aligned
pkvol_subset.loc[pkvol_subset["InstanceName"] == "Visit 2 Part 1", "InstanceName"] = 2
pkvol_subset.loc[pkvol_subset["InstanceName"] == 'Visit 1 Baseline/Screening Part 1', "InstanceName"] = 1
pkvol_subset.loc[pkvol_subset["InstanceName"] == 'Visit 3 Part 1', "InstanceName"] = 3
pkvol_subset.rename(columns = {"InstanceName" : "day"}, inplace=True)

#merge titers in
pkvol_titers = pd.merge(pkvol_subset, titers_sub, how="left", on = ["Subject", "day", "time_point"])

#total PFU
pkvol_titers["Total_PFU"] = pkvol_titers.PKVOL * pkvol_titers["PFU/mL"]

#change PCTPT8 terms
pkvol_titers.loc[pkvol_subset["PCTPT8"] == 'pre dose', "PCTPT8"] = 'pre-dose'
pkvol_titers.loc[pkvol_subset["PCTPT8"] == 'between 2 and 4 h, after start of LBP-EC01 administration (time 0)', "PCTPT8"] = 'between 2 and 4h'

pkvol_titers.to_csv("urine_pkvol_titers.csv", index=False)

In [ ]:
pkvol_titers[pkvol_titers["PCTPT8"] == "pre-dose"]["Total_PFU"].gmean()

In [ ]:
#graphing
#print(pkvol_titers.head())

#subset by day
day1 = pkvol_titers[pkvol_titers.day == 1].reset_index()
day2 = pkvol_titers[pkvol_titers.day == 2].reset_index()
day3 = pkvol_titers[pkvol_titers.day == 3].reset_index()

#day1 day 2 day 3
plt.scatter(day1.PCTPT8, day1.Total_PFU, color = 'blue')
plt.scatter(day2.PCTPT8, day2.Total_PFU, color = 'red')
plt.scatter(day3.PCTPT8, day3.Total_PFU, color = 'orange')

#mean values
day1_mean = day1.groupby('PCTPT8')["Total_PFU"].mean().reset_index()
day1_mean_reindexed = day1_mean.reindex([3,0,1,2])
day2_mean = day2.groupby('PCTPT8')["Total_PFU"].mean().reset_index()
day2_mean_reindexed = day2_mean.reindex([3,0,1,2])
day3_mean = day3.groupby('PCTPT8')["Total_PFU"].mean().reset_index()
day3_mean_reindexed = day3_mean.reindex([3,0,1,2])

print(day1_mean)
# #geometric means
# constant_replacement = 1
# day1['Total_PFU'] = day1['Total_PFU'].replace(0, constant_replacement)
# day1['Total_PFU'] = 
# day1_gmean = day1.groupby('PCTPT8')["Total_PFU"].agg(gmean).reset_index()
# print(day1_gmean)

plt.plot(day1.PCTPT8.unique(), day1_mean_reindexed["Total_PFU"], color = "blue", label = 'Day 1', marker = 'x')
plt.plot(day2.PCTPT8.unique(), day2_mean_reindexed["Total_PFU"], color = "red", label = 'Day 2', marker = 'x')
plt.plot(day3.PCTPT8.unique(), day3_mean_reindexed["Total_PFU"], color = "orange", label = 'Day 3', marker = 'x')

plt.title('Urine Total PFU')
plt.xlabel('Collection Time')
plt.ylabel('Total PFU')
plt.yscale('log')
plt.axhline(y=3.72e12, color='black', linestyle='--', label='Total PFU Dosed \n (3.7e12 PFU)')
plt.gcf().set_size_inches(7,5)
plt.legend(loc='upper right', bbox_to_anchor=(1.36,1))

plt.savefig("Urine_PFU_Balance.pdf", bbox_inches='tight')
plt.show()


In [ ]:
#graphing
#print(pkvol_titers.head())

#drop if total PFU is N/A
pkvol_titers_cleaned = pkvol_titers.dropna(subset=["Total_PFU"])

#subset by day
day1 = pkvol_titers_cleaned[pkvol_titers_cleaned.day == 1].reset_index()
day2 = pkvol_titers_cleaned[pkvol_titers_cleaned.day == 2].reset_index()
day3 = pkvol_titers_cleaned[pkvol_titers_cleaned.day == 3].reset_index()

#day1
plt.scatter(day1.PCTPT8, day1.Total_PFU, color = 'blue')
plt.scatter(day2.PCTPT8, day2.Total_PFU, color = 'red')
plt.scatter(day3.PCTPT8, day3.Total_PFU, color = 'orange')

#mean values
day1_mean = day1.groupby('PCTPT8')["Total_PFU"].mean().reset_index()
day1_mean_reindexed = day1_mean.reindex([3,0,1,2])

#geomeans
constant_replacement = 1
day1['Total_PFU'] = day1['Total_PFU'].replace(0, constant_replacement)
day1_gmean = day1.groupby('PCTPT8')['Total_PFU'].agg(gmean).reset_index()
day1_gmean_reindexed = day1_gmean.reindex([3,0,1,2])

day2['Total_PFU'] = day2['Total_PFU'].replace(0, constant_replacement)
day2_gmean = day2.groupby('PCTPT8')['Total_PFU'].agg(gmean).reset_index()
day2_gmean_reindexed = day2_gmean.reindex([3,0,1,2])

day3['Total_PFU'] = day3['Total_PFU'].replace(0, constant_replacement)
day3_gmean = day3.groupby('PCTPT8')['Total_PFU'].agg(gmean).reset_index()
day3_gmean_reindexed = day3_gmean.reindex([3,0,1,2])


plt.plot(day1.PCTPT8.unique(), day1_gmean_reindexed["Total_PFU"], color = "blue", label = 'Day 1', marker = 'x')
plt.plot(day2.PCTPT8.unique(), day2_gmean_reindexed["Total_PFU"], color = "red", label = 'Day 2', marker = 'x')
plt.plot(day3.PCTPT8.unique(), day3_gmean_reindexed["Total_PFU"], color = "orange", label = 'Day 3', marker = 'x')

plt.title('Urine PFU')
plt.xlabel('Collection Time')
plt.ylabel('Total PFU')
plt.yscale('log')
plt.ylim(1, 1e13)   # Set y-axis limits
plt.axhline(y=3.72e12, color='black', linestyle='--', label='Total PFU Dose \n 3.72e12')
plt.legend(loc="upper left", bbox_to_anchor=(1,1))

plt.savefig("Urine_PFU.pdf", bbox_inches='tight')
plt.show()